In [1]:
import re
from modules.managers.kanji_recommendation_manager import KanjiRecommendationManager
from modules.models.content_dependencies import ContentDependencies
from modules.models.study_order import StudyOrder
from modules import session
from src.utils import kanji_html
from googletrans import Translator
import gtts

s = session.Session()
s.set_user('laiscarraro')
user = s.get_user()
recommender = KanjiRecommendationManager(user)
kanji_order = recommender.get_latest_model()

unlocked_kanji = kanji_order[
    (kanji_order.Kanji.isin(
        [k for k in user.get_kanji()]
    ))
]

dependencies = ContentDependencies(user.get_unified_subtitles())
study = StudyOrder(unlocked_kanji, dependencies, kanji_order)
all_sentences = study.get_unlocked_sentences()
sentences = all_sentences.drop_duplicates(
    subset=['anime_name', 'content']
)

In [2]:
sentences.groupby(['anime_name', 'filename']).content.count()

anime_name                          filename                                           
anohana the flower we saw that day  Anohana The Flower We Saw That Day.S01E01.CC.ja.srt    6
                                    Anohana The Flower We Saw That Day.S01E02.CC.ja.srt    5
                                    Anohana The Flower We Saw That Day.S01E03.CC.ja.srt    8
                                    Anohana The Flower We Saw That Day.S01E04.CC.ja.srt    3
                                    Anohana The Flower We Saw That Day.S01E05.CC.ja.srt    3
                                                                                          ..
yuri!!! on ice                      Yuri!!! On Ice.S01E08.JA.srt                           6
                                    Yuri!!! On Ice.S01E09.JA.srt                           9
                                    Yuri!!! On Ice.S01E10.JA.srt                           5
                                    Yuri!!! On Ice.S01E11.JA.srt           

In [3]:
def findLongestConseqSubseq(arr):
    n = len(arr)
    arr = arr + [-1]
 
    ans_list = []
    lista_provisoria = []
    for i in range(1, n+1):
        if arr[i]-arr[i-1] == 1:
            lista_provisoria.append(arr[i-1])
        else:
            lista_provisoria.append(arr[i-1])
            if len(lista_provisoria) > len(ans_list):
                ans_list = lista_provisoria
            lista_provisoria = []
 
    return ans_list

In [4]:
arr = sentences[sentences.content != ''].index.tolist()
findLongestConseqSubseq(arr)

[2237, 2238, 2239]

In [5]:
sentences.loc[2237:2239]

,index,anime_name,filename,start_time,end_time,content,has_unlocked_score,has_locked_score
2237,10435.0,ousama ranking,[SubsPlease] Ousama Ranking - 10 (720p) [043EE...,0 days 00:12:02.800000,0 days 00:12:03.970000,一つ！,1.0,0.0
2238,10436.0,ousama ranking,[SubsPlease] Ousama Ranking - 10 (720p) [043EE...,0 days 00:12:06.470000,0 days 00:12:07.640000,二つ！,1.0,0.0
2239,10437.0,ousama ranking,[SubsPlease] Ousama Ranking - 10 (720p) [043EE...,0 days 00:12:09.600000,0 days 00:12:11.310000,三つ！,1.0,0.0


In [6]:
sentences['index'] = sentences.index
grouped = sentences.groupby(['anime_name', 'filename'])['index'].apply(list).reset_index()
grouped

C:\Users\Laís\AppData\Local\Temp\ipykernel_21648\1181327369.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentences['index'] = sentences.index


,anime_name,filename,index
0,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E01.CC.j...,"[94011, 94047, 94219, 94220, 94288, 94312]"
1,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E02.CC.j...,"[94327, 94351, 94435, 94475, 94685]"
2,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E03.CC.j...,"[94734, 94823, 94887, 94888, 94889, 94907, 950..."
3,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E04.CC.j...,"[95576, 95580, 95674]"
4,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E05.CC.j...,"[95958, 95960, 95962]"
...,...,...,...
211,yuri!!! on ice,Yuri!!! On Ice.S01E08.JA.srt,"[35223, 35268, 35318, 35346, 35416, 35458]"
212,yuri!!! on ice,Yuri!!! On Ice.S01E09.JA.srt,"[35657, 35678, 35705, 35769, 35784, 35799, 358..."
213,yuri!!! on ice,Yuri!!! On Ice.S01E10.JA.srt,"[36166, 36207, 36212, 36272, 36452]"
214,yuri!!! on ice,Yuri!!! On Ice.S01E11.JA.srt,"[36580, 36646, 36835]"


In [7]:
grouped['seq'] = grouped['index'].apply(findLongestConseqSubseq)
grouped['n_seq'] = grouped['seq'].apply(len)
grouped[grouped.n_seq > 1]

,anime_name,filename,index,seq,n_seq
0,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E01.CC.j...,"[94011, 94047, 94219, 94220, 94288, 94312]","[94219, 94220]",2
2,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E03.CC.j...,"[94734, 94823, 94887, 94888, 94889, 94907, 950...","[94887, 94888, 94889]",3
11,anohana the flower we saw that day,Anohana_The Flower_We Saw_That_Day_001.srt,"[90502, 90624, 90625, 90678]","[90624, 90625]",2
13,anohana the flower we saw that day,Anohana_The Flower_We Saw_That_Day_003.srt,"[91075, 91119, 91120, 91132, 91238]","[91119, 91120]",2
25,boku dake ga inai machi,data/pre_downloaded_subs/Boku_Dake_Ga_Inai_Mac...,"[43669, 43699, 43701, 43703, 43714, 43716, 437...","[43744, 43745]",2
33,boku dake ga inai machi,data/pre_downloaded_subs/Boku_Dake_Ga_Inai_Mac...,"[45943, 45973, 45974]","[45973, 45974]",2
45,free!,data/pre_downloaded_subs/Free!/Free!.S01E10.JA...,"[49982, 49983, 50151, 50228]","[49982, 49983]",2
51,haikyuu!!,data/pre_downloaded_subs/Haikyuu!!/Haikyuu!!_0...,"[104139, 104201, 104221, 104222, 104303, 10430...","[104221, 104222]",2
53,haikyuu!!,data/pre_downloaded_subs/Haikyuu!!/Haikyuu!!_0...,"[105082, 105084, 105161, 105201, 105261, 10526...","[105327, 105328]",2
54,haikyuu!!,data/pre_downloaded_subs/Haikyuu!!/Haikyuu!!_0...,"[105504, 105525, 105550, 105560, 105573, 10563...","[105730, 105731]",2


In [19]:
sequences = grouped[grouped.n_seq > 1]['seq'].apply(
    (
        lambda a: ' '.join(sentences.loc[a].content.values)
    )
)

In [24]:
grouped['sequences'] = sequences

In [25]:
grouped

,anime_name,filename,index,seq,n_seq,sequences
0,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E01.CC.j...,"[94011, 94047, 94219, 94220, 94288, 94312]","[94219, 94220]",2,そう あの日…― あの日だって…
1,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E02.CC.j...,"[94327, 94351, 94435, 94475, 94685]",[94327],1,NaN
2,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E03.CC.j...,"[94734, 94823, 94887, 94888, 94889, 94907, 950...","[94887, 94888, 94889]",3,めんますっかり大人になっててよ… お… 大人に？ （ぽっぽ）おう！ いい女だったぜ
3,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E04.CC.j...,"[95576, 95580, 95674]",[95576],1,NaN
4,anohana the flower we saw that day,Anohana The Flower We Saw That Day.S01E05.CC.j...,"[95958, 95960, 95962]",[95958],1,NaN
...,...,...,...,...,...,...
211,yuri!!! on ice,Yuri!!! On Ice.S01E08.JA.srt,"[35223, 35268, 35318, 35346, 35416, 35458]",[35223],1,NaN
212,yuri!!! on ice,Yuri!!! On Ice.S01E09.JA.srt,"[35657, 35678, 35705, 35769, 35784, 35799, 358...",[35657],1,NaN
213,yuri!!! on ice,Yuri!!! On Ice.S01E10.JA.srt,"[36166, 36207, 36212, 36272, 36452]",[36166],1,NaN
214,yuri!!! on ice,Yuri!!! On Ice.S01E11.JA.srt,"[36580, 36646, 36835]",[36580],1,NaN
